Retrieve info.json files

In [ ]:
import os
import glob

sacred_dir = "sacred_cluster"
parent_dir = os.path.dirname(os.getcwd())
info_json_files = glob.glob(
    os.path.join(parent_dir, sacred_dir, "**/*info.json"), recursive=True)
print(len(info_json_files))

Check if file can be opened, is not empty and experiment  reached at least 2millions time steps.

In [ ]:
import sys
import json
original_stdout = sys.stdout # Save a reference to the original standard output
with open('remove_command.txt', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    for i in info_json_files:
        try:
            with open(i, "r") as file:
                infos = json.load(file)
                if (len(infos.keys()) == 0):
                    print("empty  dict", i)
                if infos["test_return_mean_T"][-1] < 2000000:
                    # print("not long enough")
                    # print(i)
                    # print(infos["test_return_mean_T"][-1])
                    print("rm -r " + str(os.path.basename(os.path.dirname(i))))
                    print("rm -r models/"+str(os.path.basename(os.path.dirname(i))))
                    print("rm -r tb_logs/"+str(os.path.basename(os.path.dirname(i))))
                    print("rm -r sacred/"+str(os.path.basename(os.path.dirname(i))))

        except Exception as e:
            # print("Error")
            # print(e)
            # print(i)
            print("rm -r " + str(os.path.basename(os.path.dirname(i))))
            print("rm -r models/"+str(os.path.basename(os.path.dirname(i))))
            print("rm -r tb_logs/"+str(os.path.basename(os.path.dirname(i))))
            print("rm -r sacred/"+str(os.path.basename(os.path.dirname(i))))
    sys.stdout = original_stdout # Reset the standard output to its original value



Count the number of each experiment

In [ ]:
dict_n_exp = {}
for i in info_json_files:
    exp_name = os.path.basename(os.path.dirname(i))
    exp_name = exp_name.split("_")
    exp_name = [exp_name[0], exp_name[4], exp_name[5]]
    exp_name = "_".join(exp_name)
    dict_n_exp[exp_name] = dict_n_exp.get(exp_name, 0) + 1

for k, v in dict_n_exp.items():
    print(k, v)

In [ ]:
#Check missing experiments
methods = ["qmix", "qvmix", "ddmac", "coma", "iql"] # "dqn"
env1 = ["uc", "c"]
env2 = [3, 5, 7, 9, 10]#, 50]#, 100]
n_exp_desired = 10

remaining_exp = {"_".join([method, env1_, str(env2_)]): n_exp_desired for
                 method in methods for env1_ in env1 for env2_ in env2}
for k, v in remaining_exp.items():
    try:
        remaining_exp[k] -= dict_n_exp[k]
    except Exception as e:
        print(k,  type(e), e)

list_remaining = []
for k, v in remaining_exp.items():
    if v != 0:
        if v < 0:
            print("{}: Too much ({})".format(k, v))
        else:
            print("{} Need {} more".format(k, v))
            list_remaining.append((k,v))


In [ ]:
for i in list_remaining:
    exp = i[0].split("_")
    method = exp[0]
    env1 = exp[1]
    env2 = exp[2]
    if int(env2) <= 10:
        method_n = 10
    else:
        method_n=env2
    print("sbatch --array=1-"+str(i[1])+" launch.sbatch "+method+"_uc_"+str(method_n)+" struct_"+env1+"_"+str(env2))